In [9]:
import onnxruntime as ort
session = ort.InferenceSession("yolov10m.onnx", providers=["CPUExecutionProvider"])

这里的providers参数根据自己的实际情况设置，我使用的是GPU所以设置的是"CUDAExecutionProvider"。
如果用CPU进行推理，则需设置为"CPUExecutionProvider"；
如果有TensorRT的环境，还可以设置为"TensorrtExecutionProvider"

In [10]:
# 查看模型的输入层和输出层的属性
for input in session.get_inputs():
   print("input name: ", input.name)
   print("input shape: ", input.shape)
   print("input type: ", input.type)

for output in session.get_outputs():
   print("output name: ", output.name)
   print("output shape: ", output.shape)
   print("output type: ", output.type)

input name:  images
input shape:  [1, 3, 640, 640]
input type:  tensor(float)
output name:  output0
output shape:  [1, 300, 6]
output type:  tensor(float)


从上面的打印信息可以知道，模型有一个尺寸为[1, 3, 640, 640]的输入层和一个尺寸分别为[1, 300, 6]的输出层。

In [11]:
import cv2
import numpy as np

In [12]:
# 实现下面数据预处理的函数，preprocess
def preprocess(bgr_image, src_w, src_h, dst_w, dst_h):
   image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
   
   ratio = min(dst_w/src_w, dst_h/src_h)
   border_w = int(round(src_w * ratio / 2) * 2)
   border_h = int(round(src_h * ratio / 2) * 2)
   x_offset = (dst_w - border_w ) // 2  
   y_offset = (dst_h - border_h ) // 2

   image = cv2.resize(image, (border_w, border_h))
   image = cv2.copyMakeBorder(
       image, y_offset, y_offset, x_offset, x_offset, 
       cv2.BORDER_CONSTANT, value=(114, 114, 114)
       )

   image = image.astype(np.float32) / 255.0
   image = np.transpose(image, (2, 0, 1))
   input_tensor = np.expand_dims(image, axis=0)
   return input_tensor, ratio, x_offset, y_offset


整个数据预处理的流程如下：
上面的preprocess函数：
把OpenCV读取的BGR格式图片转换为RGB格式；
计算缩放比例和需要填充的区域，把原始图片进行等比例缩放，对不足的区域进行填充让输入图片的尺寸匹配模型的输入尺寸；
对像素值除以255做归一化操作；
把图像数据的通道顺序由HWC调整为CHW；
扩展数据维度，将数据的维度调整为NCHW。

In [ ]:
#       数据预处理 用opencv读取图片后，首先要做预处理                                                                                                                                                                                                                                                                                                                                                                                                                                                                       数据预处理
image = cv2.imread("DJI_20240616080249_0066_V.jpeg")
print("image shape: ", image.shape)
image_height, image_width, _ = image.shape
_, _, model_height, model_width = session.get_inputs()[0].shape

input_tensor, ratio, x_offset, y_offset = preprocess(image, image_width, image_height, model_width, model_height)


In [15]:
from ultralytics import YOLOv10
 
#数据集配置文件
data_yaml_path = r'D:/LMself/yolo/yolov10/coco.yaml'
#预训练模型
pre_model_name = r'D:/LMself/yolo/yolov10/ultralytics/cfg/models/v10/yolov10m.yaml'
 
if __name__ == '__main__':
    #加载预训练模型
    model = YOLOv10(pre_model_name)
    #训练生成的文件保存路径名
    savename = 'train_yolov10s'
    #训练模型
    results = model.train(data=data_yaml_path,
                          epochs=10,
                          name=savename)

New https://pypi.org/project/ultralytics/8.3.23 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.18 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=D:/LMself/yolo/yolov10/ultralytics/cfg/models/v10/yolov10m.yaml, data=D:/LMself/yolo/yolov10/coco.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train_yolov10s2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, 

100%|██████████| 755k/755k [00:09<00:00, 83.4kB/s]


                   from  n    params  module                                       arguments                     


  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics.nn.modules.block.SCDown          [384, 576, 3, 2]              
  8                  -1  2   1689984  ultralytics.nn.modules.block.C2fCIB          [576,

train: Scanning D:\LMself\yolo\datasets\train... 0 images, 8 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<00:00, 43.46it/s]

...


val: Scanning D:\LMself\yolo\datasets\val... 0 images, 7 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<00:00, 362.38it/s]

val: WARNING ⚠️ No labels found in D:\LMself\yolo\datasets\val.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: D:\LMself\yolo\datasets\val.cache
WARNING ⚠️ No labels found in D:\LMself\yolo\datasets\val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.


Plotting labels to D:\LMself\yolo\yolov10\runs\detect\train_yolov10s2\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 129 weight(decay=0.0), 142 weight(decay=0.0005), 141 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to D:\LMself\yolo\yolov10\runs\detect\train_yolov10s2
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


  0%|          | 0/1 [00:00<?, ?it/s]

: 